# Sistema de Recomendación de Contenido Netflix con Cosine Similarity

Este notebook implementa un sistema de recomendación de contenido para el catálogo de Netflix utilizando la técnica
de Similitud de Coseno (Cosine Similarity). Desarrollamos dos enfoques:
## 1. Recomendación basada en una sola variable (descripción / overview)
## 2. Recomendación basada en múltiples variables (descripción, título, director y elenco)

In [1]:
## Librerías necesarias
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# Definir la función clean_text antes de usarla
def clean_text(text):
    if isinstance(text, str):
        # Convertir a minúsculas
        text = text.lower()
        # Eliminar puntuación
        text = re.sub(f'[{string.punctuation}]', ' ', text)
        # Eliminar números
        text = re.sub(r'\d+', '', text)
        # Eliminar espacios múltiples
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return ''

In [5]:
# Cargar el dataset con el motor de Python para manejar errores de formato
try:
    df = pd.read_csv('./netflix_all_data.csv', engine='python')
except Exception as e:
    print(f"Error al cargar el archivo: {e}")
    # Intentar con skiprows si hay problemas en filas específicas
    df = pd.read_csv('netflix_all_data.csv', engine='python', on_bad_lines='skip')

# Crear una copia para no modificar el dataframe original
df_clean = df.copy()

In [7]:
# Rellenar valores nulos antes de aplicar la función
df_clean['overview'] = df_clean['overview'].fillna('')
df_clean['title'] = df_clean['title'].fillna('')
df_clean['genres'] = df_clean['genres'].fillna('')
df_clean['director'] = df_clean['director'].fillna('')
df_clean['top_cast'] = df_clean['top_cast'].fillna('')

In [9]:
# Aplicar limpieza de texto
df_clean['overview_clean'] = df_clean['overview'].apply(clean_text)
df_clean['title_clean'] = df_clean['title'].apply(clean_text)
df_clean['genres_clean'] = df_clean['genres'].apply(clean_text)
df_clean['director_clean'] = df_clean['director'].apply(clean_text)
df_clean['cast_clean'] = df_clean['top_cast'].apply(clean_text)

In [11]:
# Crear característica combinada para recomendación multi-variable
df_clean['combined_features'] = (df_clean['overview_clean'] + ' ' +
                                df_clean['title_clean'] + ' ' +
                                df_clean['genres_clean'] + ' ' +
                                df_clean['director_clean'] + ' ' +
                                df_clean['cast_clean'])

# Verificar que todo se haya creado correctamente
print(f"Número de filas en el dataset: {len(df_clean)}")
print(f"Primeras filas de características combinadas:")
print(df_clean['combined_features'].head(2))

Número de filas en el dataset: 33526
Primeras filas de características combinadas:
0    when a hostage rescue mission creates a new en...
1    an orphaned teen hits the road with a mysterio...
Name: combined_features, dtype: object


## 3. Implementación del Sistema de Recomendación con Similitud de Coseno

La similitud de coseno mide la similitud entre dos vectores calculando el coseno del ángulo entre ellos.
Esta técnica es muy útil para sistemas de recomendación basados en contenido.

### 3.1 Recomendación basada en una sola variable (Overview)

In [13]:
# Crear vectorizador TF-IDF
tfidf = TfidfVectorizer(stop_words='english')

# Aplicar TF-IDF a las descripciones
tfidf_matrix = tfidf.fit_transform(df_clean['overview_clean'])

print(f"Dimensión de la matriz TF-IDF (overview): {tfidf_matrix.shape}")

# Calcular matriz de similitud de coseno
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

print(f"Dimensión de la matriz de similitud: {cosine_sim.shape}")

# Crear mapeo de índices
indices = pd.Series(df_clean.index, index=df_clean['title']).drop_duplicates()

Dimensión de la matriz TF-IDF (overview): (33526, 21583)
Dimensión de la matriz de similitud: (33526, 33526)


In [37]:
# Función de recomendación basada en overview
def get_recommendations_overview(title, cosine_sim=cosine_sim, df=df_clean, indices=indices):
    # Obtener el índice del título
    try:
        idx = indices[title]
    except KeyError:
        return pd.DataFrame(columns=['title', 'type', 'genres', 'similarity_score'])
    
    # Obtener puntuaciones de similitud con todos los demás
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Ordenar películas/series por similitud
    # Manejar diferentes tipos de datos correctamente
    def get_score(item):
        i, score = item
        # Si es un array, obtener el primer elemento
        if hasattr(score, '__len__') and len(score) > 0:
            return score[0]
        # Si es un escalar, usarlo directamente
        return score
    
    sim_scores = sorted(sim_scores, key=get_score, reverse=True)
    
    # Obtener los 10 títulos más similares (excluyendo el propio título)
    sim_scores = sim_scores[1:11]
    
    # Obtener índices de los títulos
    movie_indices = [i[0] for i in sim_scores]
    
    # Prepara los scores para el dataframe
    scores = []
    for i, score in sim_scores:
        if hasattr(score, '__len__') and len(score) > 0:
            scores.append(float(score[0]))
        else:
            scores.append(float(score))
    
    # Devolver las 10 películas/series más similares con sus puntuaciones
    recommendations = df.iloc[movie_indices][['title', 'type', 'genres']].copy()
    recommendations['similarity_score'] = scores
    
    return recommendations

### 3.2 Recomendación basada en múltiples variables (Combined Features)

In [39]:
# Crear vectorizador TF-IDF para características combinadas
tfidf_combined = TfidfVectorizer(stop_words='english')

# Aplicar TF-IDF a las características combinadas
tfidf_matrix_combined = tfidf_combined.fit_transform(df_clean['combined_features'])

print(f"\nDimensión de la matriz TF-IDF (combinada): {tfidf_matrix_combined.shape}")

# Calcular matriz de similitud de coseno
cosine_sim_combined = cosine_similarity(tfidf_matrix_combined, tfidf_matrix_combined)



Dimensión de la matriz TF-IDF (combinada): (33526, 35143)


In [41]:
# Función de recomendación basada en características combinadas
def get_recommendations_combined(title, cosine_sim=cosine_sim_combined, df=df_clean, indices=indices):
    # Obtener el índice del título
    try:
        idx = indices[title]
    except KeyError:
        return pd.DataFrame(columns=['title', 'type', 'genres', 'director', 'similarity_score'])
    
    # Obtener puntuaciones de similitud con todos los demás
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Ordenar películas/series por similitud
    # Manejar diferentes tipos de datos correctamente
    def get_score(item):
        i, score = item
        # Si es un array, obtener el primer elemento
        if hasattr(score, '__len__') and len(score) > 0:
            return score[0]
        # Si es un escalar, usarlo directamente
        return score
    
    sim_scores = sorted(sim_scores, key=get_score, reverse=True)
    
    # Obtener los 10 títulos más similares (excluyendo el propio título)
    sim_scores = sim_scores[1:11]
    
    # Obtener índices de los títulos
    movie_indices = [i[0] for i in sim_scores]
    
    # Prepara los scores para el dataframe
    scores = []
    for i, score in sim_scores:
        if hasattr(score, '__len__') and len(score) > 0:
            scores.append(float(score[0]))
        else:
            scores.append(float(score))
    
    # Devolver las 10 películas/series más similares con sus puntuaciones
    recommendations = df.iloc[movie_indices][['title', 'type', 'genres', 'director']].copy()
    recommendations['similarity_score'] = scores
    
    return recommendations

# Función para comparar recomendaciones
def compare_recommendations(title):
    print(f"Comparación de recomendaciones para: '{title}'")
    
    # Obtener recomendaciones con ambos métodos
    recs_overview = get_recommendations_overview(title)
    recs_combined = get_recommendations_combined(title)
    
    # Mostrar títulos recomendados y sus puntuaciones
    comparison = pd.DataFrame({
        'Overview Based': recs_overview['title'].values,
        'Overview Score': recs_overview['similarity_score'].values.round(3),
        'Combined Based': recs_combined['title'].values,
        'Combined Score': recs_combined['similarity_score'].values.round(3)
    })
    
    return comparison

In [43]:
# Ejemplo de recomendación basada en características combinadas
print(f"\nRecomendaciones para '{title_example}' basadas en características combinadas:")
recommendations_combined = get_recommendations_combined(title_example)
print(recommendations_combined)


Recomendaciones para 'Counterattack' basadas en características combinadas:
                  title   type                                         genres  \
1    The Electric State  movie             Science Fiction, Adventure, Action   
2            Demon City  movie               Action, Crime, Fantasy, Thriller   
3                Amaran  movie                  Action, Drama, Adventure, War   
4           Revelations  movie                       Thriller, Crime, Mystery   
5   Plankton: The Movie  movie  Animation, Adventure, Comedy, Family, Fantasy   
6        Little Siberia  movie                        Comedy, Thriller, Drama   
7              Squad 36  movie                 Action, Crime, Drama, Thriller   
8         The Life List  movie                         Romance, Comedy, Drama   
9                   xXx  movie             Action, Adventure, Thriller, Crime   
10        The Godfather  movie                                   Drama, Crime   

                    director  s

## 4. Evaluación del Sistema de Recomendación

Comparamos las recomendaciones generadas por ambos enfoques.

In [45]:
# Función para comparar recomendaciones
def compare_recommendations(title):
    print(f"Comparación de recomendaciones para: '{title}'")

    # Obtener recomendaciones con ambos métodos
    recs_overview = get_recommendations_overview(title)
    recs_combined = get_recommendations_combined(title)

    # Mostrar títulos recomendados y sus puntuaciones
    comparison = pd.DataFrame({
        'Overview Based': recs_overview['title'].values,
        'Overview Score': recs_overview['similarity_score'].values.round(3),
        'Combined Based': recs_combined['title'].values,
        'Combined Score': recs_combined['similarity_score'].values.round(3)
    })

    return comparison

In [47]:
# Elegir un título popular para la comparación
popular_title = df_clean[df_clean['vote_count'] > 1000].sort_values('popularity', ascending=False)['title'].iloc[5]
comparison_df = compare_recommendations(popular_title)
print("\nComparación de recomendaciones:")
print(comparison_df)

Comparación de recomendaciones para: 'Law & Order: Special Victims Unit'

Comparación de recomendaciones:
        Overview Based  Overview Score       Combined Based  Combined Score
0   The Electric State           0.097   The Electric State           0.052
1           Demon City           0.097           Demon City           0.052
2               Amaran           0.097               Amaran           0.052
3          Revelations           0.097          Revelations           0.052
4  Plankton: The Movie           0.097  Plankton: The Movie           0.052
5       Little Siberia           0.097       Little Siberia           0.052
6             Squad 36           0.097             Squad 36           0.052
7        The Life List           0.097        The Life List           0.052
8                  xXx           0.097                  xXx           0.052
9        The Godfather           0.097        The Godfather           0.052


In [49]:
# Calcular el porcentaje de coincidencia entre ambos métodos
def calculate_overlap(title):
    recs_overview = get_recommendations_overview(title)['title'].tolist()
    recs_combined = get_recommendations_combined(title)['title'].tolist()

    # Calcular superposición
    overlap = set(recs_overview).intersection(set(recs_combined))
    overlap_percentage = len(overlap) / 10 * 100

    return overlap_percentage, list(overlap)

overlap_pct, common_titles = calculate_overlap(popular_title)
print(f"\nPorcentaje de coincidencia entre métodos: {overlap_pct:.1f}%")
print(f"Títulos comunes: {common_titles}")


Porcentaje de coincidencia entre métodos: 100.0%
Títulos comunes: ['Little Siberia', 'The Godfather', 'The Life List', 'Squad 36', 'Plankton: The Movie', 'xXx', 'Revelations', 'The Electric State', 'Amaran', 'Demon City']


## 5. Recomendaciones por Género

Implementamos una variante que filtra recomendaciones por género.

In [81]:
def get_recommendations_by_genre(title, genre, cosine_sim=cosine_sim_combined, df=df_clean, indices=indices):
    # Obtener el índice del título
    try:
        idx = indices[title]
        # Asegurarse de que idx sea un escalar
        if hasattr(idx, '__iter__') and not isinstance(idx, str):
            idx = idx.iloc[0] if hasattr(idx, 'iloc') else idx[0]
    except KeyError:
        return pd.DataFrame(columns=['title', 'type', 'genres', 'similarity_score'])
    
    # Obtener puntuaciones de similitud
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Filtrar por género (el género debe estar presente en la columna genres)
    genre_matches = []
    for i, score in sim_scores:
        if isinstance(df.iloc[i]['genres'], str) and genre.lower() in df.iloc[i]['genres'].lower():
            genre_matches.append((i, score))
    
    # Ordenar por similitud
    def get_score(item):
        i, score = item
        if hasattr(score, '__len__') and len(score) > 0:
            return score[0]
        return score
    
    genre_matches = sorted(genre_matches, key=get_score, reverse=True)
    
    # Excluir el título original
    idx_int = int(idx)
    genre_matches = [match for match in genre_matches if int(match[0]) != idx_int]
    
    # Preparar los datos para las recomendaciones
    titles_seen = set()
    unique_recommendations = []
    scores = []
    
    # Filtrar duplicados por título
    for i, score in genre_matches:
        title = df.iloc[i]['title']
        if title not in titles_seen:
            titles_seen.add(title)
            unique_recommendations.append(i)
            
            # Extraer puntuación
            if hasattr(score, '__len__') and len(score) > 0:
                scores.append(float(score[0]))
            else:
                scores.append(float(score))
                
            # Limitar a 10 recomendaciones únicas
            if len(unique_recommendations) >= 10:
                break
    
    # Si no hay suficientes coincidencias, devolver lo que tengamos
    if len(unique_recommendations) == 0:
        return pd.DataFrame(columns=['title', 'type', 'genres', 'similarity_score'])
    
    # Devolver recomendaciones únicas
    recommendations = df.iloc[unique_recommendations][['title', 'type', 'genres']].copy()
    recommendations['similarity_score'] = scores
    
    return recommendations

In [83]:
# Ejemplo de recomendación por género
genre_example = "thriller"
print(f"\nRecomendaciones para '{popular_title}' en el género '{genre_example}':")
recommendations_genre = get_recommendations_by_genre(popular_title, genre_example)
print(recommendations_genre)


Recomendaciones para 'Law & Order: Special Victims Unit' en el género 'thriller':
                         title   type  \
17637                Cold Eyes  movie   
7                     Squad 36  movie   
17070              Untraceable  movie   
12626        The Pale Blue Eye  movie   
17038                 S.W.A.T.  movie   
2156            Den of Thieves  movie   
7795   The Roundup: No Way Out  movie   
14921                  Khufiya  movie   
16277                      '71  movie   
15119                Vengeance  movie   

                                        genres  similarity_score  
17637                  Crime, Action, Thriller          0.124684  
7               Action, Crime, Drama, Thriller          0.121530  
17070  Thriller, Crime, Mystery, Drama, Horror          0.081530  
12626         Thriller, Crime, Horror, Mystery          0.077011  
17038                  Action, Thriller, Crime          0.071778  
2156                   Action, Crime, Thriller          0.06961

## 6. Guardar el Modelo Entrenado

Guardamos los vectorizadores TF-IDF y las matrices de similitud para su uso en producción.

In [ ]:
# Guardar vectorizadores
with open('tfidf_vectorizer_overview.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

with open('tfidf_vectorizer_combined.pkl', 'wb') as f:
    pickle.dump(tfidf_combined, f)

# No guardamos las matrices de similitud completas debido a su tamaño
# En producción, calcularemos la similitud bajo demanda

print("\nModelos guardados exitosamente.")

## 7. Conclusiones

En este notebook hemos implementado dos enfoques para un sistema de recomendación de contenido Netflix:

1. **Recomendación basada en una sola variable (overview)**:
   - Enfoque simple que considera solo las descripciones
   - Ideal para recomendaciones rápidas con recursos limitados
   - Menor precisión contextual

2. **Recomendación basada en múltiples variables**:
   - Enfoque más robusto que integra título, descripción, director y elenco
   - Proporciona recomendaciones más contextualizadas
   - Requiere más recursos computacionales

Ambos enfoques utilizan la técnica de similitud de coseno (Cosine simmilarity) que es efectiva para capturar la semántica del contenido.
El sistema puede ser desplegado como parte de una aplicación web o API para proporcionar recomendaciones personalizadas a los usuarios.

In [ ]:
#Vectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings('ignore')

# Descargar recursos de NLTK (ejecutar la primera vez)
# nltk.download('punkt')
# nltk.download('stopwords')

## 1. Carga y Exploración de Datos

Cargamos el dataset de Netflix procesado y exploramos sus características principales.

In [71]:
# Cargar dataset
df = pd.read_csv('netflix_all_data.csv', low_memory=False)

# Explorar las primeras filas
print("Muestra de datos:")
df.head(2)

# Información general del dataset
print("\nInformación del dataset:")
df.info()

# Resumen estadístico
print("\nResumen estadístico:")
df.describe()

# Distribución de tipos de contenido
print("\nDistribución por tipo de contenido:")
print(df['type'].value_counts())

# Verificar valores nulos en columnas relevantes
print("\nValores nulos en columnas relevantes:")
null_cols = ['title', 'overview', 'genres', 'director', 'top_cast']
print(df[null_cols].isnull().sum())

Muestra de datos:

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33526 entries, 0 to 33525
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    33526 non-null  int64  
 1   title                 33526 non-null  object 
 2   original_title        33526 non-null  object 
 3   overview              33342 non-null  object 
 4   popularity            33526 non-null  float64
 5   vote_average          33526 non-null  float64
 6   vote_count            33526 non-null  int64  
 7   release_date          33525 non-null  object 
 8   genres                33421 non-null  object 
 9   production_countries  33266 non-null  object 
 10  original_language     33526 non-null  object 
 11  runtime               33526 non-null  int64  
 12  type                  33526 non-null  object 
 13  seasons               15274 non-null  float64
 14  episodes              1527

## 2. Preprocesamiento de Datos

Preparamos los datos para el sistema de recomendación, manejando valores nulos y creando características combinadas.

In [67]:
# Función para limpiar texto
def clean_text(text):
    if isinstance(text, str):
        # Convertir a minúsculas
        text = text.lower()
        # Eliminar puntuación
        text = re.sub(f'[{string.punctuation}]', ' ', text)
        # Eliminar números
        text = re.sub(r'\d+', '', text)
        # Eliminar espacios múltiples
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return ''

# Crear copia para no modificar el dataframe original
df_clean = df.copy()

# Rellenar valores nulos
df_clean['overview'] = df_clean['overview'].fillna('')
df_clean['title'] = df_clean['title'].fillna('')
df_clean['genres'] = df_clean['genres'].fillna('')
df_clean['director'] = df_clean['director'].fillna('')
df_clean['top_cast'] = df_clean['top_cast'].fillna('')

# Aplicar limpieza de texto
df_clean['overview_clean'] = df_clean['overview'].apply(clean_text)
df

,id,title,original_title,overview,popularity,vote_average,vote_count,release_date,genres,production_countries,...,seasons,episodes,director,top_cast,production_companies,available_regions,region_count,global_title,discovery_method,release_year
0,1356039,Counterattack,Contraataque,When a hostage rescue mission creates a new en...,180.8217,8.442,558,2025-02-27,"Action, Adventure, Thriller",Mexico,...,NaN,NaN,Chava Cartas,"Luis Alberti, Noé Hernández, Leonardo Alonso",Draco Films,"US, GB, CA, AU, FR, DE, ES, IT, NL, SE, DK, NO...",41,True,region_EG,2025.0
1,777443,The Electric State,The Electric State,An orphaned teen hits the road with a mysterio...,141.7648,6.600,821,2025-03-07,"Science Fiction, Adventure, Action",United States of America,...,NaN,NaN,"Joe Russo, Anthony Russo","Millie Bobby Brown, Chris Pratt, Ke Huy Quan","AGBO, Skybound Entertainment","US, GB, CA, AU, FR, DE, ES, IT, NL, SE, DK, NO...",41,True,region_EG,2025.0
2,1405338,Demon City,Demon City 鬼ゴロシ,Framed for his family's murder and left for de...,129.1596,6.900,176,2025-02-26,"Action, Crime, Fantasy, Thriller",Japan,...,NaN,NaN,Seiji Tanaka,"Toma Ikuta, Masahiro Higashide, Miou Tanaka","Netflix, AMUSE, Digital Frontier","US, GB, CA, AU, FR, DE, ES, IT, NL, SE, DK, NO...",41,True,region_EG,2025.0
3,927342,Amaran,அமரன்,A heroic true story of Major Mukund Varadaraja...,92.4649,7.400,197,2024-10-31,"Action, Drama, Adventure, War",India,...,NaN,NaN,Rajkumar Periasamy,"Sivakarthikeyan, Sai Pallavi, Rahul Bose","Raajkamal Films International, Sony Pictures I...","US, GB, CA, AU, DE, ES, IT, NL, SE, DK, NO, FI...",39,True,region_EG,2024.0
4,1210938,Revelations,계시록,A pastor who believes in divine revelation and...,90.3278,6.300,62,2025-03-20,"Thriller, Crime, Mystery",South Korea,...,NaN,NaN,Yeon Sang-ho,"Ryu Jun-yeol, Shin Hyun-been, Shin Min-jae","Wow Point, Esperanto Filmoj","US, GB, CA, AU, FR, DE, ES, IT, NL, SE, DK, NO...",41,True,region_EG,2025.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33521,100043,Bloodride,Blodtur,The doomed passengers aboard a spectral bus he...,1.2632,6.900,192,2020-03-13,"Sci-Fi & Fantasy, Mystery, Action & Adventure",NO,...,1.0,6.0,NaN,NaN,NaN,"US, GB, CA, AU, FR, DE, ES, IT, NL, SE, DK, NO...",40,True,language_no,2020.0
33522,223785,Billionaire Island,Milliardærøya,The ruthless owner of a Norwegian fish farming...,1.1858,6.800,17,2024-09-12,"Comedy, Drama",NO,...,1.0,6.0,NaN,"Trine Wiggen, Svein Roger Karlsen, Kåre Conradi",Rubicon TV AS,"US, GB, CA, AU, FR, DE, ES, IT, NL, SE, DK, NO...",41,True,language_no,2024.0
33523,129933,Post Mortem: No One Dies in Skarnes,Post Mortem: Ingen dør i Skarnes,Live Hallangen is declared dead but hours late...,1.1854,7.300,64,2021-08-25,"Drama, Comedy, Mystery",NO,...,1.0,6.0,NaN,"Kathrine Thorborg Johansen, André Sørum, Elias...",NaN,"US, GB, CA, AU, FR, DE, ES, IT, NL, SE, DK, NO...",41,True,language_no,2021.0
33524,285545,Frie tøyler,Frie tøyler,NaN,0.4087,0.000,0,2025-03-17,Documentary,NO,...,1.0,4.0,NaN,NaN,NaN,"US, GB, CA, AU, FR, DE, ES, IT, NL, SE, DK, NO...",40,True,language_no,2025.0
